1 Quick Dogs v Cats

In [ ]:
# git pull https://github.com/fastai.git
# cd fastai
# conda env create -f environment.yml
# source activate fastai
# python -m ipykernel install --user --name fastai --display-name "Python 3 (FastAI)"

In [2]:
import sys; import os
sys.path.insert(1, os.path.join(os.getcwd()+'/fastai/fastai/'))

In [1]:
from fastai.conv_learner import *
PATH = 'data/'
sz=224; bs=64

In [ ]:
tfms = tfms_from_model(resnet50, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)
learn = ConvLearner.pretrained(resnet50, data)

%time learn.fit(1e-2, 3, cycle_len=1) # 3 cycles of len 1 at 1e-2

In [ ]:
learn.unfreeze()
learn.bn_freeze(True) # important for large models --> stops updates of BN moving avgs
                      # use for models w/ ~ > 34 layers, & larger imgs & reg.s of intrst
%time learn.fit([1e-5, 1e-4, 1e-2], 1, cycle_len=1)

In [ ]:
%time log_preds, y = learn.TTA()

metrics.log_loss(y, np.exp(log_preds)), accuracy(log_preds, y)